In [128]:
import pandas as pd
import math

class Irt:
    """Classe creata per applicare algoritmi di Item Response Theory ad oggetti che rappresentano le prove d'esame,
       i quali hanno una riga per studente e le colonne rappresentano gli item su cui applicare le funzioni."""
    
    class RangeError(Exception):
       """Eccezione per input fuori dal range"""
       pass
    
    __minVal = 0
    __maxVal = 10
        
    def __init__(self, path):        
        """Il costruttore prende come parametro il path del file csv da analizzare."""        
        self.__df = pd.read_csv(path, index_col = 0) # rendo la prima colonna l'indice della tabella
        self.numItems = len(self.__df.columns) # salvo il numero di items    
        self.__dfProb = pd.DataFrame(columns=self.__df.columns, index=self.__df.index)
        
    def __getNumInput(self,msg):
        """Prendo valore in input assicurandomi che sia intero e compreso nell'intervallo"""
        while True:
            try:
                n = float(input(msg))
                if n < self.__minVal or n > self.__maxVal:
                    raise RangeError
                break
            except ValueError:
                print("L'input non è un numero")
            except RangeError:
                print("L'input non è compreso nell'intervallo (", self.__minVal, "," , self.__maxVal, ")")
        return n
    
    def changeInterval(self, min, max):
        """Modifica l'intervallo di valori delle difficoltà e dei discriminanti"""
        self.__minVal = min
        self.__maxVal = max     
        
    def __getProbInput(self,msg):
        """Prendo valore in input assicurandomi che sia una probabilità, quindi compreso tra lo 0% ed il 100%"""
        while True:
            try:
                n = float(input(msg))
                if n < 0 or n > 100:
                    raise RangeError
                break
            except ValueError:
                print("L'input non è un numero")
            except RangeError:
                print("L'input non è una probabilità compresa tra 0 e 100")
        return n      
       
    def __getDiff(self, n):
        """Metodo usato per ottenere la lista di difficoltà degli esercizi"""
        
        lsDiff = pd.DataFrame(columns = self.__df.columns , index = ["Difficulty"])
        print("Insersci la difficoltà per i seguenti item, essa deve essere compresa nell'intervallo (", self.__minVal, "," , self.__maxVal, ") : \n")
        for columns in lsDiff:
            lsDiff.loc["Difficulty",columns] = self.__getNumInput(columns + ": ")
        return lsDiff 

    def __getDiscrim(self, n):
        """Metodo usato per ottenere la lista di discriminanti degli esercizi"""        
        
        lsDiscrim = pd.DataFrame(columns = self.__df.columns , index = ["Discriminant"])
        print("Insersci il discriminante per i seguenti item, essa deve essere compresa nell'intervallo (", self.__minVal, "," , self.__maxVal, ") : \n")
        for columns in lsDiscrim:
            lsDiscrim.loc["Discriminant",columns] = self.__getNumInput(columns + ": ")
        return lsDiscrim

    def __getGuess(self, n):
        """Metodo usato per ottenere la lista di probabilità di indovinare esercizi"""    
        
        lsGuess = pd.DataFrame(columns = self.__df.columns , index = ["Guess"])
        print("Insersci la difficoltà per i seguenti item, essa deve essere compresa tra 0 e 100: \n")
        for columns in lsGuess:
            lsGuess.loc["Guess",columns] = self.__getProbInput(columns + ": ")/100
        return lsGuess  
    
    def __estimateAbility(self, lsDiff):
        """Metodo usato per fornire una stima delle abilità degli studenti"""   
        
        lsAb = pd.DataFrame(index = self.__df.index , columns = ["Ability"])
        difficulties = lsDiff.loc["Difficulty"]        
        for index, row in self.__df.iterrows():
            ability = (row*difficulties).sum()/difficulties.sum() # stimo mediante media ponderata
            lsAb.loc[index,"Ability"] = round(ability, 2)
        return lsAb      
    
    def pl1(self):
        """Metodo usato per applicare il modello PL1 dell'Item Response Theory, esso usa solo il parametro delle difficoltà""" 
        
        lsDiff = self.__getDiff(self.numItems)
        lsAb   = self.__estimateAbility(lsDiff)

        for index, row in self.__dfProb.iterrows():
            for columns in self.__dfProb:
                ability    = lsAb.loc  [index,"Ability"]
                difficulty = lsDiff.loc["Difficulty",columns]
                res = (math.exp(ability - difficulty)) / (1 + math.exp(ability - difficulty))
                self.__dfProb.loc[index, columns] = str(round(res * 100 , 2)) + "%"
        return self.__dfProb

    def pl2(self):
        """Metodo usato per applicare il modello PL1 dell'Item Response Theory, esso usa le difficoltà ed i discriminanti"""  
        
        lsDiff    = self.__getDiff(self.numItems)
        lsAb      = self.__estimateAbility(lsDiff)
        print()
        lsDiscrim = self.__getDiscrim(self.numItems)

        for index, row in self.__dfProb.iterrows():
            for columns in self.__dfProb:
                ability      = lsAb.loc     [index,"Ability"]
                difficulty   = lsDiff.loc   ["Difficulty",columns]      
                discriminant = lsDiscrim.loc["Discriminant",columns]
                res = (math.exp(discriminant * (ability - difficulty))) / (1 + math.exp(discriminant * (ability - difficulty)))
                self.__dfProb.loc[index, columns] = str(round(res * 100 , 2)) + "%"
        return self.__dfProb

    def pl3(self):
        """Metodo usato per applicare il modello PL1 dell'Item Response Theory, esso usa le difficoltà, discriminanti e 
        le probabilità di indovinare gli esercizi"""        
        
        lsDiff   = self.__getDiff(self.numItems)
        lsAb     = self.__estimateAbility(lsDiff)
        print()
        lsDiscrim= self.__getDiscrim(self.numItems)
        print()
        lsGuess  = self.__getGuess(self.numItems)
        
        for index, row in self.__dfProb.iterrows():
            for columns in self.__dfProb:
                ability      = lsAb.loc     [index,"Ability"]
                difficulty   = lsDiff.loc   ["Difficulty",columns]      
                discriminant = lsDiscrim.loc["Discriminant",columns]            
                guessProb    = lsGuess.loc  ["Guess",columns]
                res = guessProb + (1 - guessProb) * (math.exp(discriminant * (ability - difficulty))) / (1 + math.exp(discriminant * (ability - difficulty)))
                self.__dfProb.loc[index, columns] = str(round(res * 100 , 2)) + "%"
        return self.__dfProb    
    
    def evalDf(self):
        """Questo metodo ha lo scopo di stabilire quanto sono corretti i parametri stabiliti per il calcolo delle probabilità"""
        
        dfDiff = pd.DataFrame(columns=self.__df.columns, index=self.__df.index)
        dfCorrect = pd.DataFrame(columns=self.__df.columns, index=["Precise"])
        for index, row in dfDiff.iterrows():
            for columns in dfDiff:
                votePerc = (self.__df.loc[index, columns] * 100)/self.__maxVal 
                correctProb = float((self.__dfProb.loc[index, columns])[:-1])
                dfDiff.loc[index, columns] = abs(votePerc - correctProb) # faccio la differenza tra la prob e la percentuale del voto rispetto al massimo
        for columns in dfCorrect:
            res = 100 - dfDiff.mean()[columns] # calcolo la media delle differenze per trovare la percentuale media di errore e poi la sottraggo a 100 per indicare la precisione
            dfCorrect.loc["Precise", columns] = str(round(res, 2)) + "%"
        return dfCorrect    
    
    def getDf(self):
        """Restituisce il la tabella relativa al file csv utilizzato"""
        return self.__df    

In [129]:
test = Irt("testcsv.csv")
x = test.pl1()

Insersci la difficoltà per i seguenti item, essa deve essere compresa nell'intervallo ( 0 , 10 ) : 

Esercizio 1: 2
Esercizio 2: 4
Esercizio 3: 6
Esercizio  4: 7
Esercizio  5: 8


In [130]:
test.evalDf()

,Esercizio 1,Esercizio 2,Esercizio 3,Esercizio 4,Esercizio 5
Precise,75.59%,83.47%,87.17%,66.95%,70.61%
